# 함수 및 라이브러리 기본 세팅

In [ ]:
import pandas as pd

from bs4  import BeautifulSoup
from time import sleep


from selenium                          import webdriver
from webdriver_manager.chrome          import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.common.exceptions        import NoSuchElementException

In [ ]:
def get_players_data_by_game(players):
    """
    players: 요소가 선수명으로 이루어진 리스트
    
    이 함수는 statiz에서 각 선수가 뛴 모든 국내 시즌에서의 기록들을
    연도 별 일 단위로 가져옵니다.
    """
    all_players = {}

    driver  = webdriver.Chrome(executable_path=ChromeDriverManager().install())
    exception_players = {
        "김현수": "http://www.statiz.co.kr/player.php?opt=3&name=김현수&birth=1988-01-12",
        "김상수": "http://www.statiz.co.kr/player.php?opt=3&name=김상수&birth=1990-03-23",
        "박병호": "http://www.statiz.co.kr/player.php?opt=3&name=박병호&birth=1986-07-10",
        "박건우": "http://www.statiz.co.kr/player.php?opt=3&name=박건우&birth=1990-09-08",
    }
    for name in players:
        if name in exception_players:
            url = exception_players[name]
            driver.get(url)
            sleep(1)
        else:
            url = f'http://www.statiz.co.kr/player.php?opt=3&name={name}'
            driver.get(url)
            sleep(1)

        # 팀 이름, 선수 이름, 포지션
        player_info     = driver.find_element_by_class_name('callout').text
        player_team     = player_info.split('\n')[-1].split(',')[0]
        player_position = player_info.split(', ')[1]
            
        print(player_team, name, player_position)
        
        # 투수는 타자 기록이 없기 때문에 넘어간다.
        if player_position == '투수':
            print(f"{name}은(는) 투수입니다 !")
            continue

        seasons_wrapper = driver.find_element_by_xpath('/html/body/div[1]/div[1]/div/section[2]/div/div[2]/div/div[2]/div')
        seasons = seasons_wrapper.find_elements_by_tag_name('a')
        
        # 과거부터 최신 순으로
        years = [season.text for season in seasons]
        years.sort(reverse=False)
        
        player_data = {}
        for year in years:
            driver.get(url + f"&year={year}")
            sleep(1)

            html       = BeautifulSoup(driver.page_source, 'lxml')
            table_html = html.find_all("table")[2]
            table      = pd.read_html(str(table_html), encoding='cp949')[0]

            # table 정보 추가 및 기본 전처리
            table["이름"]  = name
            table["포지션"] = player_position
            table["팀"]    = player_team
            table = table.rename(columns={table.columns[0]: "경기일"})
            
            # 경기일 column에서 이상한 row 삭제 및 index 재정렬
            drop_index = table["경기일"][table["경기일"].apply(lambda x: x.find('월')) > 0].index
            table = table.drop(drop_index).reset_index(drop=True)
            table["경기일"] = table["경기일"].apply(lambda x: f"{year}-{x}")
            table["경기일"] = table["경기일"].astype('datetime64')
            table["연도"]  = table["경기일"].dt.year
            
            # 필요없는 column drop
            drop_cols = ["경기일", "포지션", "상대"]
            table = table.drop(drop_cols, axis=1)
            
            # 결과에 대해 전처리
            table["결과"] = table["결과"].apply(lambda x: 1 if x[0] == "W" else 0)

            player_data[year] = table

        all_players[name] = player_data

    driver.quit()
    return all_players


def group_by_day(df, day):
    """
    df : 데이터 프레임
    day: 묶을 경기의 단위
    
    이 함수는 데이터 프레임 row들을 day만큼 묶어서 합산하여 각각의 row로 묶어서
    데이터 프레임 형태로 반환합니다.
    """
    group_day = int(len(df) / day)
    
    bundle_by_day = pd.DataFrame()
    for i in range(group_day):
        start_idx = day * i
        end_idx   = day * (i + 1)
        temp_df = df.loc[start_idx:end_idx, :]
        
        int_cols = [
            '결과', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점',
            '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비'
        ]
        float_cols = [
            '타율', '출루', '장타', 'OPS', 
            '투구', 'avLI', 'RE24', 'WPA'
        ]
        
        # int col과 float col 연산
        # 하나의 row로 만든다.
        temp_df[int_cols]   = temp_df[int_cols].astype('int')
        temp_df[float_cols] = temp_df[float_cols].astype('float')
        
        
        temp_df.loc[start_idx, int_cols]   = temp_df[int_cols].sum()
        temp_df.loc[start_idx, float_cols] = temp_df[float_cols].mean()
        temp_df.loc[start_idx, "P"]        = temp_df["P"].value_counts().sort_values().index[-1]
        temp_df.loc[start_idx, "타순"]      = temp_df["타순"].value_counts().sort_values().index[-1]
        
        bundle_by_day = bundle_by_day.append(temp_df.loc[start_idx, :])
        
    return bundle_by_day.reset_index(drop=True)


def get_player_data_a_year(all_player_data, players, year, division=8):
    """
    all_player_data : 선수 이름을 key 값으로 가지고 있으며, 그 안에 year와, dataframe을 key와 value로 가지고 있는 dictionary
    players         : 가져오고자 하는 선수명의 리스트
    year            : 가져오고자 하는 데이터의 연도
    division        : 경기묶음의 단위
    선수의 특정 연도 데이터를 division 수의 경기만큼 묶어 반환합니다.
    """
    all_df_by_day = pd.DataFrame()
    for player in players:
        if year in all_player_data[player]:
            df        = all_player_data[player][year]
            df_by_day = group_by_day(df, division)
            all_df_by_day = all_df_by_day.append(df_by_day).reset_index(drop=True)

    return all_df_by_day

# 데이터 가져오기

## 프리미어 15 선수 시즌 데이터 가져오기

In [ ]:
premier_15         = pd.read_csv("data/15_premier.csv")
main_idx           = premier_15["타수"].sort_values(ascending=False)[:9].index
premier_15         = premier_15.loc[main_idx].reset_index(drop=True)
premier_15_players = premier_15["이름"].tolist()
premier_year       = "2015"

# 크롤링 가져오기
premier_15_player_kbo_data = get_players_data_by_game(premier_players)
premier_15_player_kbo_data = get_player_data_a_year(premier_15_player_kbo_data, premier_players, premier_year, division=8)

# 백업 df 생성
backup_15_premier = premier_15_player_kbo_data.copy()

## 19연도 kbo, premier 데이터 가져오기

In [ ]:
pd.options.display.max_columns = 999

premier_2019       = pd.read_csv("data/19_premier.csv")
sub_idx            = premier_2019["타수"].sort_values()[:4].index
premier_2019       = premier_2019.drop(sub_idx, axis=0).reset_index(drop=True)
premier_19_players = premier_2019["이름"].tolist()
premier_year       = "2019"

# 크롤링 하기
premier_19_player_kbo_data = get_players_data_by_game(premier_players)
premier_19_player_kbo_data = get_player_data_a_year(premier_19_player_kbo_data, premier_players, premier_year, division=8)

# 백업 df 생성
backup_19_premier = premier_19_player_kbo_data.copy()

## 21년도 kbo 시즌 데이터 가져오기

In [ ]:
# 투수 포지션을 제외한 도쿄 올림픽 멤버
tokyo_year    = "2021"
tokyo_players = [
    "박건우", "김현수", "이정후", "박해민", "강백호",
    "황재균", "오지환", "김혜성", "양의지", "강민호",
    "오재일", "박민우", "최주환", "허경민"
]

# 크롤링 하기
tokyo_player_kbo_data = get_players_data_by_game(tokyo_players)
tokyo_player_kbo_data = get_player_data_a_year(tokyo_player_kbo_data, tokyo_players, tokyo_year, division=8)

# 백업 df 생성
backup_tokyo = tokyo_player_kbo_data.copy()

# 변수명 변경

In [ ]:
premier_15_player_kbo_data = backup_15_premier.copy()
premier_19_player_kbo_data = backup_19_premier.copy()
# tokyo_player_kbo_data   = backup_tokyo.copy()

columns = [
    'RES', 'ORD', 'P', 'SP', 'AB', 'R', 'H', '2B', '3B', 'HR',
    'TB', 'RBI', 'SB', 'CS', 'BB', 'HBP', 'IBB', 'SO', 'GDP',
    'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'PN',
    'avLI', 'RE24', 'WPA', 'PLAYER', 'TEAM', 'YEAR'
]

# 컬럼명 변경
premier_15_player_kbo_data.columns = columns
premier_19_player_kbo_data.columns = columns
# tokyo_player_kbo_data.columns   = columns

# tokyo_player_kbo_data.head(3)
premier_15_player_kbo_data.head(3)
premier_19_player_kbo_data.head(3)

In [ ]:
def add_default_derived_var(df):
    df['B']    = df['H'] - (df['2B'] + df['3B'] + df['HR'])
    df['GPA']  = (df.OBP * 1.8 + df.SLG) / 4
    df['BB/K'] = df.BB / df.SO
    df['ISOP'] = df.SLG - df.AVG
    df['XBH']  = df['2B'] + df['3B'] + df.HR


# 기본 파생변수 추가
add_default_derived_var(premier_15_player_kbo_data)
# add_default_derived_var(premier_19_player_kbo_data)
# add_default_derived_var(tokyo_player_kbo_data)
    
# 실수 변수들 가져오기
float_cols = premier_15_player_kbo_data.dtypes[premier_15_player_kbo_data.dtypes == "float"].index.tolist()
# float_cols = tokyo_player_kbo_data.dtypes[tokyo_player_kbo_data.dtypes == "float"].index.tolist()

# 실수 변수들 3째 자리까지 표기
premier_player_kbo_data[float_cols] = premier_15_player_kbo_data[float_cols].apply(round, args=(3, ))
# tokyo_player_kbo_data[float_cols]   = tokyo_player_kbo_data[float_cols].apply(round, args=(3, ))

premier_15_player_kbo_data.head(3)
# tokyo_player_kbo_data.head(3)

In [ ]:
cols = [
    'PLAYER', 'TEAM', 'YEAR', 'RES', 'ORD', 'P', 'SP', 'AB', 'R', 
    'H', 'B', '2B', '3B', 'HR', 'XBH', 'TB', 'RBI',
    'SB', 'CS', 'BB', 'HBP', 'IBB', 'SO', 'GDP', 'SH', 'SF', 'AVG', 'OBP',
    'SLG', 'ISOP', 'OPS','GPA','PN', 'avLI', 'RE24', 'WPA'
]
premier_15_player_kbo_data = premier_15_player_kbo_data[cols]
# tokyo_player_kbo_data   = tokyo_player_kbo_data[cols]

# csv 파일 저장

In [ ]:
premier_15_path = "data/2015_premier_player_season.csv"
premier_19_path = "data/2019_premier_player_season.csv"
# tokyo_path   = "data/2021_tokyo_player_season.csv"

premier_player_kbo_data.to_csv(premier_15_path, index=False)
# premier_player_kbo_data.to_csv(premier_19_path, index=False)
# tokyo_player_kbo_data.to_csv(tokyo_path, index=False)

In [ ]:
pd.read_csv(premier_15_path)

In [ ]:
pd.read_csv(premier_19_path)

In [ ]:
pd.read_csv(tokyo_path)